In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
from tqdm import tqdm


def get_data(link,start_date, end_date, page):
    formatted_link = link.format(start_date.strftime('%d%%2F%m%%2F%Y'), end_date.strftime('%d%%2F%m%%2F%Y'), page)
    print(formatted_link)
    response = requests.get(formatted_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def get_table_from_soup(soup):
    # Encontrar todas las filas de las tablas
    rows = soup.findAll('tr')

    # Inicializar listas vacías para almacenar los datos
    data = []

    # Iterar sobre cada fila de la tabla
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Limpiar espacios y caracteres especiales
        if len(cols) == 4:  # Solo extraer si hay 4 columnas
            data.append(cols)

    # Crear un DataFrame de pandas
    df = pd.DataFrame(data, columns=['#', 'Parlamentar', 'Voto', 'Obs'])

    return df

In [3]:
link_parts = [
    'https://www25.senado.leg.br/web/atividade/materias?p_p_id=materia_WAR_atividadeportlet',
    'p_p_lifecycle=0',
    '_materia_WAR_atividadeportlet_pesquisaAvancada=true',
    '_materia_WAR_atividadeportlet_ano=',
    '_materia_WAR_atividadeportlet_tipoAutor=SENADOR',
    '_materia_WAR_atividadeportlet_dataApresentacaoInicial={}',
    '_materia_WAR_atividadeportlet_numero=',
    '_materia_WAR_atividadeportlet_situacaoTramitacao=TRAMITACAO_ENCERRADA',
    '_materia_WAR_atividadeportlet_dataApresentacaoFinal={}',
    '_materia_WAR_atividadeportlet_anoNorma=',
    '_materia_WAR_atividadeportlet_codigoClasseTematica=',
    '_materia_WAR_atividadeportlet_palavraChave=',
    '_materia_WAR_atividadeportlet_tipoNorma=',
    '_materia_WAR_atividadeportlet_codigoParlamentar=',
    '_materia_WAR_atividadeportlet_numeroNorma=',
    '_materia_WAR_atividadeportlet_nomeRelator=',
    '_materia_WAR_atividadeportlet_btnSubmit=',
    '_materia_WAR_atividadeportlet_nomeAutor=',
    '_materia_WAR_atividadeportlet_codigoTipoConteudo=',
    '_materia_WAR_atividadeportlet_siglasTipos=DESTPEC%2CDESTPLORD',
    '_materia_WAR_atividadeportlet_p={}'
]

link = '&'.join(link_parts)

In [4]:
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2024, 5, 1)
page=1 # Start at 1

document_links = {}
while True:
    # print(link)
    soup = get_data(link, start_date, end_date, page)
    # print(soup.prettify())
    # break
    result = soup.find('div', class_='sf-proposicoes-resultado div-zebra')
    # print(result.prettify())
    if result is None:
        break
    for div in result.children:
        pl_link = div.find('a')
        if pl_link != -1:
            # print(pl_link.text)
            # print(pl_link['href'])
            document_links[pl_link.text] = pl_link['href']
    page += 1
    print(page)
    # if page > 2:
    #     break

https://www25.senado.leg.br/web/atividade/materias?p_p_id=materia_WAR_atividadeportlet&p_p_lifecycle=0&_materia_WAR_atividadeportlet_pesquisaAvancada=true&_materia_WAR_atividadeportlet_ano=&_materia_WAR_atividadeportlet_tipoAutor=SENADOR&_materia_WAR_atividadeportlet_dataApresentacaoInicial=01%2F01%2F2020&_materia_WAR_atividadeportlet_numero=&_materia_WAR_atividadeportlet_situacaoTramitacao=TRAMITACAO_ENCERRADA&_materia_WAR_atividadeportlet_dataApresentacaoFinal=01%2F05%2F2024&_materia_WAR_atividadeportlet_anoNorma=&_materia_WAR_atividadeportlet_codigoClasseTematica=&_materia_WAR_atividadeportlet_palavraChave=&_materia_WAR_atividadeportlet_tipoNorma=&_materia_WAR_atividadeportlet_codigoParlamentar=&_materia_WAR_atividadeportlet_numeroNorma=&_materia_WAR_atividadeportlet_nomeRelator=&_materia_WAR_atividadeportlet_btnSubmit=&_materia_WAR_atividadeportlet_nomeAutor=&_materia_WAR_atividadeportlet_codigoTipoConteudo=&_materia_WAR_atividadeportlet_siglasTipos=DESTPEC%2CDESTPLORD&_materia_WAR

In [11]:
all_data = pd.DataFrame()
for key, value in tqdm(document_links.items()):
    doc_link = value + '/votacoes'
    response = requests.get(doc_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    alert_portlet_body = soup.find('div', class_='alert alert-warning')
    if alert_portlet_body is not None:
        continue
    table_df = get_table_from_soup(soup)
    table_df['KEY'] = key
    table_df['LINK'] = doc_link
    all_data = pd.concat([all_data, table_df])
all_data.to_csv('votacoes.csv', index=False)

100%|██████████| 1843/1843 [21:47<00:00,  1.41it/s]


In [ ]:
assert False

In [ ]:
# document_links = {
#     "asdas":"https://www25.senado.leg.br/web/atividade/materias/-/materia/140554"
# }

In [5]:
    

    


import random

while True:
    random_pick = random.choice(list(document_links.keys()))
    document_link = document_links[random_pick] + '/votacoes'
    print(random_pick)
    response = requests.get(document_link)
    # print(document_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(soup.prettify())
    alert_portlet_body = soup.find('div', class_='alert alert-warning')
    if alert_portlet_body is None:
        # print('No warning! Continuing...')
        print(document_link)
        # table_content = soup.findAll('table', class_='table table-striped table-condensed')
        # for table in table_content:
        #     print(table.prettify())
        # break

PL 486/2021
https://www25.senado.leg.br/web/atividade/materias/-/materia/149766/votacoes
PL 2048/2020
PL 1342/2020
PL 229/2022
PL 835/2020
PL 1986/2021
PL 5217/2020
PL 4279/2023
PL 5213/2020
PL 5096/2020
PL 3144/2021
PL 2721/2023
https://www25.senado.leg.br/web/atividade/materias/-/materia/157538/votacoes
PL 3892/2020
https://www25.senado.leg.br/web/atividade/materias/-/materia/143597/votacoes
PL 1856/2020
PL 2384/2023
https://www25.senado.leg.br/web/atividade/materias/-/materia/158907/votacoes
PL 2291/2023
PLN 35/2023
PL 3387/2020
PL 3535/2023
PL 229/2022
PL 1481/2021
PEC 32/2022 (fase 2)
https://www25.senado.leg.br/web/atividade/materias/-/materia/155594/votacoes
PL 1856/2021
PL 4059/2021
https://www25.senado.leg.br/web/atividade/materias/-/materia/152805/votacoes
PL 1808/2021
PL 1610/2020
PL 5494/2020
PL 1741/2022
PL 835/2020
PL 4550/2021
PL 2253/2022
https://www25.senado.leg.br/web/atividade/materias/-/materia/154451/votacoes
PLN 7/2020
PLN 41/2021
PL 3607/2020
PL 556/2022
PL 5382/

KeyboardInterrupt: 

In [ ]:
import pandas as pd

In [ ]:
# Encontrar todas las filas de las tablas
rows = soup.findAll('tr')

# Inicializar listas vacías para almacenar los datos
data = []

# Iterar sobre cada fila de la tabla
for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]  # Limpiar espacios y caracteres especiales
    if len(cols) == 4:  # Solo extraer si hay 4 columnas
        data.append(cols)

# Crear un DataFrame de pandas
df = pd.DataFrame(data, columns=['#', 'Parlamentar', 'Voto', 'Obs'])

# Mostrar el DataFrame
print(df)

     #              Parlamentar Voto             Obs
0    1             Acir Gurgacz  Sim                
1    2        Alessandro Vieira  Sim                
2    3              Alvaro Dias    -  Não Compareceu
3    4           Angelo Coronel  Sim                
4    5        Antonio Anastasia  Sim                
..  ..                      ...  ...             ...
76  77  Veneziano Vital do Rêgo  Sim                
77  78      Wellington Fagundes  Sim                
78  79                 Weverton  Sim                
79  80             Zenaide Maia  Sim                
80  81         Zequinha Marinho  Sim                

[81 rows x 4 columns]


In [ ]:
for document_title, document_link in document_links.items():
    response = requests.get(document_link)
    print(document_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    conteudo_projeto = soup.find('div', class_='accordion')
    children = conteudo_projeto.children
    children = list(children)
    print(children[2])
    # print(soup.prettify())
    break

https://www.congressonacional.leg.br/materias/pesquisa/-/materia/140430
 


In [ ]:
assert False

AssertionError: 

In [ ]:
print(len(document_links))

36


In [ ]:
for document_link in document_links:
    print(document_link)
    # response = requests.get(document_link)
    # soup = BeautifulSoup(response.text, 'html.parser')
    # print(soup.prettify())
    # break

https://www.congressonacional.leg.br/materias/pesquisa/-/materia/140430
https://www25.senado.leg.br/web/atividade/materias/-/materia/140818
https://www25.senado.leg.br/web/atividade/materias/-/materia/140817
https://www25.senado.leg.br/web/atividade/materias/-/materia/140751
https://www25.senado.leg.br/web/atividade/materias/-/materia/140717
https://www25.senado.leg.br/web/atividade/materias/-/materia/140740
https://www25.senado.leg.br/web/atividade/materias/-/materia/140741
https://www25.senado.leg.br/web/atividade/materias/-/materia/140763
https://www25.senado.leg.br/web/atividade/materias/-/materia/140637
https://www25.senado.leg.br/web/atividade/materias/-/materia/140636
https://www25.senado.leg.br/web/atividade/materias/-/materia/140629
https://www25.senado.leg.br/web/atividade/materias/-/materia/140613
https://www25.senado.leg.br/web/atividade/materias/-/materia/140562
https://www25.senado.leg.br/web/atividade/materias/-/materia/140558
https://www25.senado.leg.br/web/atividade/ma

In [ ]:
assert False, 'End of the line'

AssertionError: End of the line

In [ ]:
# List of all dates in this year until today
import datetime
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2021, 1, 1)
page=74 # Start at 1
import requests
from bs4 import BeautifulSoup

# '02%2F02%2F2024'
# '03%2F02%2F2024'
page = 1

formatted_link = link.format(start_date.strftime('%d%%2F%m%%2F%Y'), end_date.strftime('%d%%2F%m%%2F%Y'), page)
response = requests.get(formatted_link)
soup = BeautifulSoup(response.text, 'html.parser')
divs = soup.find_all('dl', class_='dl-horizontal borda-pontilhada-base')
print('NUMBER', len(divs))
while len(divs) > 0:
    print('PAGE', page)
    page += 1
    formatted_link = link.format(start_date.strftime('%d%%2F%m%%2F%Y'), end_date.strftime('%d%%2F%m%%2F%Y'), page)
    response = requests.get(formatted_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    divs = soup.find_all('dl', class_='dl-horizontal borda-pontilhada-base')
    print('NUMBER', len(divs))

# for day in range((end_date - start_date).days):
#     date = start_date + datetime.timedelta(days=day)
#     next_day = date + datetime.timedelta(days=1)
#     formatted_link = link.format(date.strftime('%d%%2F%m%%2F%Y'), next_day.strftime('%d%%2F%m%%2F%Y'))
    
    

NUMBER 10
PAGE 1
NUMBER 10
PAGE 2
NUMBER 10
PAGE 3
NUMBER 10
PAGE 4
NUMBER 10
PAGE 5
NUMBER 10
PAGE 6
NUMBER 10
PAGE 7
NUMBER 10
PAGE 8
NUMBER 10
PAGE 9
NUMBER 10
PAGE 10
NUMBER 10
PAGE 11
NUMBER 10
PAGE 12
NUMBER 10
PAGE 13
NUMBER 10
PAGE 14
NUMBER 10
PAGE 15
NUMBER 10
PAGE 16
NUMBER 10
PAGE 17
NUMBER 10
PAGE 18
NUMBER 10
PAGE 19
NUMBER 10
PAGE 20
NUMBER 10
PAGE 21
NUMBER 10
PAGE 22
NUMBER 10
PAGE 23
NUMBER 10
PAGE 24
NUMBER 10
PAGE 25
NUMBER 10
PAGE 26
NUMBER 10
PAGE 27
NUMBER 10
PAGE 28
NUMBER 10
PAGE 29
NUMBER 10
PAGE 30
NUMBER 10
PAGE 31
NUMBER 10
PAGE 32
NUMBER 10
PAGE 33
NUMBER 10
PAGE 34
NUMBER 10
PAGE 35
NUMBER 10
PAGE 36
NUMBER 10
PAGE 37
NUMBER 10
PAGE 38
NUMBER 10
PAGE 39
NUMBER 10
PAGE 40
NUMBER 10
PAGE 41
NUMBER 10
PAGE 42
NUMBER 10
PAGE 43
NUMBER 10
PAGE 44
NUMBER 10
PAGE 45
NUMBER 10
PAGE 46
NUMBER 10
PAGE 47
NUMBER 10
PAGE 48
NUMBER 10
PAGE 49
NUMBER 10
PAGE 50
NUMBER 10
PAGE 51
NUMBER 10
PAGE 52
NUMBER 10
PAGE 53
NUMBER 10
PAGE 54
NUMBER 10
PAGE 55
NUMBER 10
PAGE 56
N

In [ ]:
print(soup.prettify())

<!DOCTYPE html>
<html $container_height="" class="aui ltr" dir="ltr" lang="pt-BR">
 <head>
  <title>
   Resultado da pesquisa de matérias - Pesquisas - Senado Federal
  </title>
  <link href="https://www25.senado.leg.br/senado-theme/images/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="https://www25.senado.leg.br/senado-theme/images/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="https://www25.senado.leg.br/senado-theme/images/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="https://www25.senado.leg.br/senado-theme/images/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
  <link href="https://www25.senado.leg.br/senado-theme/images/favicon/favicon.ico" rel="shortcut icon"/>
  <link href="https://www25.senado.leg.br/senado-theme/images/favicon/site.webmanifest" rel="manifest"/>
  <meta content="#00aba9" name="msapplication-TileColor"/>
  <meta content="https://www25.senado.leg.b